# 5. Exploratory Data Analysis (EDA)


In [1]:
import pandas as pd
import sqlite3
import prettytable
from pathlib import Path

import helpers as hlp


%load_ext sql

## Setup


In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

INPUT_FILE = hlp.DATA_DIR / Path("spacex.csv")
DB_FILE = hlp.DATA_DIR / Path("05_spacex_data.sqlite3")

prettytable.DEFAULT = "DEFAULT"

## Open database


In [3]:
con = sqlite3.connect(DB_FILE)
cur = con.cursor()

## Save data to sqlite database


In [4]:
df = pd.read_csv(INPUT_FILE, encoding="utf-8", header=0, index_col=False)
df.to_sql("SPACEXTBL", con, if_exists="replace", index=False, method="multi")

101

## Connect to database


In [5]:
%sql sqlite:///data/05_spacex_data.sqlite3

## Run queries


In [6]:
# MISC. Get the name of all columns
%sql PRAGMA table_info(SPACEXTBL);

 * sqlite:///data/05_spacex_data.sqlite3
Done.


cid,name,type,notnull,dflt_value,pk
0,Date,TEXT,0,None,0
1,Time (UTC),TEXT,0,None,0
2,Booster_Version,TEXT,0,None,0
3,Launch_Site,TEXT,0,None,0
4,Payload,TEXT,0,None,0
5,PAYLOAD_MASS__KG_,INTEGER,0,None,0
6,Orbit,TEXT,0,None,0
7,Customer,TEXT,0,None,0
8,Mission_Outcome,TEXT,0,None,0
9,Landing_Outcome,TEXT,0,None,0


In [7]:
# Unique launch sites
%sql SELECT DISTINCT Launch_Site AS site FROM SPACEXTBL;

 * sqlite:///data/05_spacex_data.sqlite3
Done.


site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [8]:
# Unique boosters
%sql SELECT DISTINCT Customer AS customer FROM SPACEXTBL WHERE customer LIKE '%NASA (CRS)%';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


customer
NASA (CRS)
"NASA (CRS), Kacific 1"


In [9]:
# Payload mass carried by boosters launched by NASA (CRS)
%sql SELECT sum(PAYLOAD_MASS__KG_) AS nasa_payload_mass FROM SPACEXTBL WHERE customer LIKE '%NASA (CRS)%';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


nasa_payload_mass
48213


In [10]:
# Average payload mass carried by booster version F9 v1.1
%sql SELECT avg(PAYLOAD_MASS__KG_) AS avg_f9_payload_mass FROM SPACEXTBL WHERE Booster_Version LIKE '%F9 v1.1%';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


avg_f9_payload_mass
2534.6666666666665


In [11]:
# first successful landing on a ground pad
%sql SELECT min(Date) AS first_landing FROM SPACEXTBL WHERE Landing_Outcome = 'Success (ground pad)';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


first_landing
2015-12-22


In [12]:
%%sql
--# boosters successfully landed in dron ship and payload mass between 4000 and 6000
SELECT DISTINCT Booster_Version FROM SPACEXTBL
    WHERE 4000 < PAYLOAD_MASS__KG_ < 6000
    AND Landing_Outcome = 'Success (drone ship)';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


Booster_Version
F9 FT B1021.1
F9 FT B1022
F9 FT B1023.1
F9 FT B1026
F9 FT B1029.1
F9 FT B1021.2
F9 FT B1029.2
F9 FT B1036.1
F9 FT B1038.1
F9 B4 B1041.1


In [13]:
%%sql
-- # total number of successful and unsuccessful mission outcomes
SELECT COUNT(Mission_Outcome) FROM SPACEXTBL
    WHERE Mission_Outcome = 'Success'
    OR Mission_Outcome LIKE 'Failure%';

 * sqlite:///data/05_spacex_data.sqlite3
Done.


COUNT(Mission_Outcome)
99


In [14]:
%%sql 
-- # boosters that have carried the maximum payload mass
SELECT Booster_Version FROM SPACEXTBL
WHERE PAYLOAD_MASS__KG_ = (SELECT max(PAYLOAD_MASS__KG_) FROM SPACEXTBL);

 * sqlite:///data/05_spacex_data.sqlite3
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [15]:
%%sql
-- # month_names, failure landing_outcomes in drone whip, boosters, and launch_site for the months in year 2015
SELECT substr(date, 6, 2) As month, Booster_Version, Launch_Site FROM SPACEXTBL
WHERE Landing_Outcome LIKE '%Failure (drone ship)%'
AND CAST(substr(date, 0, 5) AS INTEGER) == 2015;

 * sqlite:///data/05_spacex_data.sqlite3
Done.


month,Booster_Version,Launch_Site
01,F9 v1.1 B1012,CCAFS LC-40
04,F9 v1.1 B1015,CCAFS LC-40


In [16]:
%%sql
-- # count of landing outcomes between 2010-06-04 and 2017-03-20 in descending order
SELECT Landing_Outcome, COUNT(Landing_Outcome) AS outcome_count FROM SPACEXTBL
GROUP BY Landing_Outcome
ORDER BY outcome_count DESC;

 * sqlite:///data/05_spacex_data.sqlite3
Done.


Landing_Outcome,outcome_count
Success,38
No attempt,21
Success (drone ship),14
Success (ground pad),9
Failure (drone ship),5
Controlled (ocean),5
Failure,3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


In [17]:
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing_Outcome'],
      dtype='object')

In [18]:
cur.close()
con.close()